# Cifar-10 이미지 분류 - v2
- 데이터 부풀리기 적용

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 15s 0us/step


In [3]:
# X_data : scaling
X_train = X_train/255.
X_test = X_test/255.

In [4]:
# y_data : onehot encoding
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

- 모델 정의/설정/학습/평가

In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import RMSprop

In [6]:
model = Sequential([
    Conv2D(32, (3,3), input_shape = X_train.shape[1:], padding="same", activation = "relu"),
    Conv2D(32, (3,3), activation = "relu"),
    MaxPooling2D(),
    Dropout(0.25),
    Conv2D(64, (3,3), padding = "same", activation = "relu"),
    Conv2D(32, (3,3), activation = "relu"),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation = "relu"),
    Dropout(0.5),
    Dense(10, activation = "softmax"),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 32)        18464     
                                                        

In [7]:
model.compile(optimizer = RMSprop(learning_rate=0.0001, decay=1e-6),
              loss = "categorical_crossentropy", metrics = ["accuracy"])

In [8]:
model_path = "best-cifar10-v2.h5"
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=10)

- 데이터 부풀리기

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, stratify = y_train, test_size = 0.2, random_state = seed
)

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range = 15,
    width_shift_range = 0.1,
    shear_range = 0.5,
    height_shift_range = 0.1,
    zoom_range = [0.8, 1.5],
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = "nearest"
)

In [11]:
datagen.fit(X_train)

- 데이터 부풀리기 한 결과로 학습

In [12]:
hist = model.fit(
    datagen.flow(X_train, y_train, batch_size = 32),
    validation_data = (X_valid, y_valid),
    epochs = 100, callbacks = [mc,es]
)

Epoch 1/100
1248/1250 [============================>.] - ETA: 0s - loss: 2.0086 - accuracy: 0.2443
Epoch 1: val_loss improved from inf to 1.83146, saving model to best-cifar10-v2.h5
1250/1250 [==============================] - 40s 21ms/step - loss: 2.0083 - accuracy: 0.2445 - val_loss: 1.8315 - val_accuracy: 0.3303
Epoch 2/100
1250/1250 [==============================] - ETA: 0s - loss: 1.8041 - accuracy: 0.3268
Epoch 2: val_loss improved from 1.83146 to 1.68445, saving model to best-cifar10-v2.h5
1250/1250 [==============================] - 26s 21ms/step - loss: 1.8041 - accuracy: 0.3268 - val_loss: 1.6845 - val_accuracy: 0.3866
Epoch 3/100
1248/1250 [============================>.] - ETA: 0s - loss: 1.7303 - accuracy: 0.3652
Epoch 3: val_loss improved from 1.68445 to 1.67372, saving model to best-cifar10-v2.h5
1250/1250 [==============================] - 27s 22ms/step - loss: 1.7302 - accuracy: 0.3653 - val_loss: 1.6737 - val_accuracy: 0.3920
Epoch 4/100
1250/1250 [==================

In [1]:
y_acc = hist.history["accuracy"]
y_vloss = hist.history["val_loss"]
xs = np.arange(1, len(y_acc) + 1)

plt.figure(figsize = (12,8))
plt.plot(xs, y_acc, label = "Accuracy")
plt.plot(xs, y_vloss, label = "Validation loss")
plt.legend(), plt.grid(), plt.xlabel("Epoch"), plt.title("MNIST CNN model v1")
plt.show()

NameError: ignored